<html>
<font color = green size = 6>
<b>
Recommendation Engine
</b>
</font>
</html>

<html>
<font color = Purple size = 4>
<b> Set up the Pyspark environment required for running the model</b>
</font>
</html>

In [1]:
#Pyspark setup
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import NumericType
from pyspark.sql.functions import *

In [2]:
sc = SparkContext.getOrCreate()
spark = SQLContext(sc)

<html>
<font color = Red size = 6>
<b> Feature Engineering</b>
</font>
</html>

<html>
<font color = blue>
Load cleaned up log files as a Spark dataframe.
</font>
</html>

In [6]:
newlogs_df = spark.read.csv(path = "newlogs.csv", header = True,inferSchema = True)

<html>
<font color = blue>
Data is in minutes for each AvatarID. So, first count AvatarIDs by grouping them into minutes.
</font>
</html>

In [7]:
df2 = newlogs_df.groupBy(['AvatarID','Guild','QueryTime']).count()

<html>
<font color = blue>
Strip time from Query Time.
</font>
</html>

In [8]:
df2 = df2.withColumn('Date', df2['QueryTime'].substr(1,9))

<html>
<font color = blue>
Now, take count of each player by grouping them by Avatar ID and Guild.
</font>
</html>

In [27]:
PlayerbyGuild = df2.groupBy(['AvatarID','Guild']).agg({"AvatarID": "count"})

In [28]:
PlayerbyGuild = PlayerbyGuild.withColumnRenamed('count(AvatarID)', 'GamesPlayed')

In [32]:
PlayerbyGuild.drop_duplicates()

122427

In [66]:
GuildData = PlayerbyGuild.groupby(['Guild']).agg({"GamesPlayed": "sum"})

In [68]:
GuildData = GuildData.withColumnRenamed('sum(GamesPlayed)','GuildGamesPlayed')

<html>
<font color = blue>
Convert Spark dataframe to Pandas.
</font>
</html>

In [55]:
import pandas as pd

In [33]:
PlayerinGuild = PlayerbyGuild.toPandas()

In [69]:
GuildGames = GuildData.toPandas()

In [43]:
matrix = PlayerinGuild.pivot(index = 'AvatarID', columns = 'Guild', values = 'GamesPlayed').fillna(0)

In [52]:
matrix.drop(u' ',axis=1, inplace=True)

<html>
<font color = Red size = 6>
<b> Build Recommendation Model - userbased collaboarative filtering </b>
</font>
</html>

In [149]:
class GuildRecommendar:
      #Remove duplicate records 
        def remove_duplicates(self,inputdata):
            initial_rows = inputdata.shape[0]
            print('Initial dataframe shape {0}'.format(inputdata.shape))
            inputdata = inputdata.drop_duplicates(['AvatarID', 'Guild'])
            current_rows = inputdata.shape[0]
            print('New dataframe shape {0}'.format(inputdata.shape))
            print('Removed {0} rows'.format(initial_rows - current_rows))
        
        def recommend_Guild(self,usertdataset,guilddataset,user_index,neighbors):
            from scipy.sparse import csr_matrix
            from sklearn.neighbors import NearestNeighbors
            wide_user_data_less_sparse = csr_matrix(usertdataset.values)
            model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
            model_knn.fit(wide_user_data_less_sparse)
            distances, indices = model_knn.kneighbors(usertdataset.iloc[user_index, :].values.reshape(1, -1), n_neighbors = neighbors)
            
            total = []
            for i in usertdataset.columns:
                for j in range(1,neighbors):
                    rowname = usertdataset.iloc[indices[0][j]].name
                    if usertdataset.loc[usertdataset.iloc[indices[0][0]].name][i] == 0.0:
                        if usertdataset.loc[rowname][i] > 0.0:
                            totalplay = guilddataset.loc[guilddataset.Guild == i, 'GuildGamesPlayed'].values
                            weight = ((1/distances[0][j])*usertdataset.loc[rowname][i])/totalplay
                            total.append({'guild': i, 'weight': weight})
            df = pd.DataFrame(total)
            df.drop_duplicates(['guild'],keep='first',inplace=True)
            df.weight = df.weight.astype(float)
            df = df.sort_values(['weight'],ascending = False)
            df = df[['guild']]
            return df
        
        #Pick a random user to provide as input in recommendation functions
        def pickrandomuser(self,usertdataset):
            import numpy as np
            query_index = np.random.choice(usertdataset.shape[0])
            return query_index
        
        #Pick a random guild to provide as input in recommendation functions
        def pickrandomguild(self,usertdataset):
            import numpy as np
            guildid = np.random.choice(usertdataset.shape[1])
            guild = usertdataset.columns[guildid]
            return guild
        
        #Execute this function to check if a user will play in the guild
        def willuserplay(self,user_index,guild,usertdataset,guilddataset,neighbors=6):
            df2 = self.recommend_Guild(usertdataset,guilddataset,user_index,neighbors)
            if (df2.guild==guild).any():
                print ('It is very likely that this user will play if he joins Guild: %s' % str(guild))
            else:
                print ('It is not likely that this user will play if he joins Guild: %s' % str(guild))          
         
        

In [143]:
gurec = GuildRecommendar()

In [135]:
gurec.pickrandomguild(matrix)

u'268'

In [136]:
gurec.pickrandomuser(matrix)

4224

In [148]:
gurec.recommend_Guild(matrix,GuildGames,65436,6)

,guild
1,370
2,9
0,21


In [151]:
gurec.willuserplay(65436,'268',matrix,GuildGames,6)

It is not likely that this user will play if he joins Guild: 268


In [150]:
gurec.willuserplay(65436,'370',matrix,GuildGames,6)

It is very likely that this user will play if he joins Guild: 370


<html>
<font color = blue size = 4>
This recommendation engine will suggest a suitable guild for the user to join and play and thus avoid player churning. <br />
</font>
</html>